---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    
    
    return spam_data[spam_data.target == 1].shape[0] / spam_data.shape[0] * 100

In [4]:
print(answer_one())
print(747/(4825+747)*100)
spam_data['target'].value_counts()

13.406317300789663
13.406317300789663


0    4825
1     747
Name: target, dtype: int64

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    # fit CountVectorizer
    vect = CountVectorizer().fit(X_train)
    
    # transform training data into matrix
    # X_train_vectorized = vect.transform(X_train)
    
    maxlen = 0
    maxlen = max([len(w) for w in vect.get_feature_names()])
    
    return [w for w in vect.get_feature_names() if len(w) == maxlen]

In [6]:
answer_two()

['com1win150ppmx3age16subscription']

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    # create Multinomial Naive Bayes classifier
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)
    
    predictions = model.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions) 

In [8]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    # fit and transform Tfidf vectorizer on X_train
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    feature_names = np.array(vect.get_feature_names())
    
    sorted_list = np.argsort(X_train_vectorized.toarray()).flatten()[::-1]
    
    top_20 = feature_names[sorted_list][:20]
    
    lowest_20 = feature_names[sorted_list][:-21:-1]
    
    # series = [(vect.get_feature_names()[sorted_tfidf_index], X_train_vectorized[sorted_tfidf_index])]
    
    return (top_20, lowest_20)

In [ ]:
answer_four()

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [17]:
def answer_five():
    from sklearn.feature_extraction.text import TfidfVectorizer
    
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)
    
    predictions = model.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)

In [18]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [11]:
def answer_six():
    avg_notspam = np.mean([len(w) for w in spam_data['text'] if spam_data.loc[spam_data['text'] == w, 'target'].iloc[0] == 0])
    avg_spam = np.mean([len(w) for w in spam_data['text'] if spam_data.loc[spam_data['text'] == w, 'target'].iloc[0] == 1])
    
    return (avg_notspam, avg_spam)

In [12]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [5]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [23]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_seven():
    
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    train_matrix = add_feature(X_train_vectorized, [len(w) for w in X_train])
    
    model_svc = SVC(C=10000)
    model_svc.fit(train_matrix, y_train)
    
    test_matrix = add_feature(vect.transform(X_test), [len(x) for x in X_test])
    predictions = model_svc.predict(test_matrix)
    
    return roc_auc_score(y_test, predictions) #Your answer here

In [24]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [32]:
def answer_eight():
    not_spam_avgnumlen = np.mean([len([int(c) for c in w if c.isdigit()]) for w in spam_data['text'] if spam_data.loc[spam_data['text'] == w, 'target'].iloc[0] == 0])
    spam_avgnumlen = np.mean([len([int(c) for c in w if c.isdigit()]) for w in spam_data['text'] if spam_data.loc[spam_data['text'] == w, 'target'].iloc[0] == 1])
    
    return (not_spam_avgnumlen, spam_avgnumlen)

In [33]:
answer_eight()

(0.29927461139896372, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

def number_of_digits(x):
    return len([[int(c) for c in w if c.isdigit()] for w in x])

def answer_nine():
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    temp_matrix = add_feature(X_train_vectorized, [len(w) for w in X_train])
    train_matrix = add_feature(temp_matrix, [number_of_digits(w) for w in X_train])
    
    model = LogisticRegression(C=100)
    model.fit(train_matrix, y_train)
    
    temp_test_matrix = add_feature(vect.transform(X_test), [len(w) for w in X_test])
    test_matrix = add_feature(temp_test_matrix, [number_of_digits(w) for w in X_test])
    predictions = model.predict(test_matrix)
    
    return roc_auc_score(y_test, predictions)

In [54]:
answer_nine()

0.96616895574079431

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [6]:
def answer_ten():
    import re
    notspam_notword = np.mean([len(re.findall(r"!\w.*", w)) for w in spam_data['text'] if spam_data.loc[spam_data['text'] == w, 'target'].iloc[0] == 0])
    spam_notword = np.mean([len(re.findall(r"!\w.*", w)) for w in spam_data['text'] if spam_data.loc[spam_data['text'] == w, 'target'].iloc[0] == 1])
    
    return (notspam_notword, spam_notword)

In [7]:
answer_ten()

(0.0062176165803108805, 0.041499330655957165)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [8]:
import re
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

def number_of_digits(x):
    return len([[int(c) for c in w if c.isdigit()] for w in x])

def number_of_nonwords(x):
    return len([re.findall(r"!\w.*", w) for w in x])

def answer_eleven():
    # vectorizing
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    # adding additional features
    temp_matrix1 = add_feature(X_train_vectorized, [len(w) for w in X_train])
    temp_matrix2 = add_feature(temp_matrix1, [number_of_digits(w) for w in X_train])
    train_matrix = add_feature(temp_matrix2, [number_of_nonwords(w) for w in X_train])
    
    # creating the model
    model = LogisticRegression(C=100)
    model.fit(train_matrix, y_train)
    
    # test model
    X_test_vectorized = vect.transform(X_test)
    temp_test_matrix1 = add_feature(X_test_vectorized, [len(w) for w in X_test])
    temp_test_matrix2 = add_feature(temp_test_matrix1, [number_of_digits(w) for w in X_test])
    test_matrix = add_feature(temp_test_matrix2, [number_of_nonwords(w) for w in X_test])
    
    predictions = model.predict(test_matrix)
    
    # auc score
    AUC = roc_auc_score(y_test, predictions)
    
    # feature_names = ['text', 'is_spam', 'length_of_doc', 'digit_count', 'non_word_char_count']
    
    feature_names = np.array(vect.get_feature_names())
    
    sorted_coef_index = model.coef_[0].argsort()
    
    # 10 largest coefficients
    smallest_coefs = feature_names[sorted_coef_index[:10]]
    
    # 10 smallest coefficients
    largest_coefs = feature_names[sorted_coef_index[:-11:-1]]
    
    return (AUC, smallest_coefs, largest_coefs)

In [9]:
answer_eleven()

(0.97927737127141212,
 array(['..', ' y', ' i', '? ', ':)', '. ', ' m', ' go', 'he', ' h'], 
       dtype='<U5'),
 array([' 0', ' 1', '50', '00', '08', '15', '80', '0p', '0 ', 'mob'], 
       dtype='<U5'))